In [ ]:
# Automação da Verificação do SEI usando Python e Selenium

In [94]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [95]:
# Passo 1 - selecionar o site
navegador.get("https://www.sei.mg.gov.br/sei/controlador_externo.php?acao=usuario_externo_logar&id_orgao_acesso_externo=0")

# Passo 2 - preencher campo de usuário e senha
navegador.find_element('xpath', '//*[@id="txtEmail"]').send_keys('SEU EMAIL AQUI') #MUDAR O USUÁRIO
navegador.find_element('xpath', '//*[@id="pwdSenha"]').send_keys('SUA SENHA AQUI') #MUDAR A SENHA

# Passo 3 - clicar no botão e entrar
navegador.find_element('xpath', '//*[@id="sbmLogin"]').click()

In [96]:
urls_processos = []

while True:
    # Localize todos os links dos processos na página atual
    elementos_links_processos = navegador.find_elements(By.XPATH, '//*[@id="tblDocumentos"]//td[2]/a')
    
    # Se não houver mais links de processos, saia do loop
    if not elementos_links_processos:
        break
    
    # Crie uma lista de URLs dos processos da página atual e adicione à lista principal
    urls_processos += [elemento.get_attribute('href') for elemento in elementos_links_processos]
    
    # Vá para a próxima página
    try:
        # Encontre o elemento de paginação
        paginacao = navegador.find_element(By.ID, 'lnkInfraProximaPaginaSuperior')
        # Clique no link para acessar a próxima página
        paginacao.click()
        # Espere um momento para a nova página carregar completamente
        time.sleep(3)
    except NoSuchElementException:
        # Se não houver mais páginas, saia do loop
        break

informacoes = []

# Iterar sobre as URLs dos processos
for url_processo in urls_processos:
    navegador.get(url_processo)
    
    # Coleta o número de processo diretamente da página
    numero_processo_pagina = navegador.find_element(By.XPATH, '//*[@id="tblCabecalho"]/tbody/tr[2]/td[2]').text
    
    # Coleta informações da tabela com id "tblCabecalho"
    tabela_cabecalho = navegador.find_element(By.ID, 'tblCabecalho')
    
    # Coleta as informações das células especificadas dentro da tabela "tblCabecalho"
    dado1 = tabela_cabecalho.find_element(By.XPATH, '//*[@id="tblCabecalho"]/tbody/tr[3]/td[2]').text
    dado2 = tabela_cabecalho.find_element(By.XPATH, '//*[@id="tblCabecalho"]/tbody/tr[4]/td[2]').text
    
    # Coleta informações da tabela com id "tblDocumentos"
    tabela_documentos = navegador.find_element(By.ID, 'tblDocumentos')
    
    # Coleta as informações da última linha da tabela "tblDocumentos"
    elementos_ultima_linha = tabela_documentos.find_elements(By.XPATH, '//*[@id="tblDocumentos"]/tbody/tr[last()]/td')
    
    # Coleta as informações específicas da última linha
    dado3 = elementos_ultima_linha[2].text
    dado4 = elementos_ultima_linha[3].text
    dado5 = elementos_ultima_linha[4].text
    
    # Armazene essas informações em uma lista
    informacoes.append({'Número de Processo Página': numero_processo_pagina, 'Tipo de Processo': dado1, 'Data de Criação': dado2, 'Último Documento': dado3, 'Data de Última Movimentação': dado4, 'Unidade do Documento': dado5})

df = pd.DataFrame(informacoes, columns=['Número de Processo Página', 'Tipo de Processo', 'Data de Criação', 'Último Documento', 'Data de Última Movimentação', 'Unidade do Documento'])

# Salve o DataFrame em um arquivo Excel
df.to_excel("dados_processos.xlsx", index=False)